<a href="https://colab.research.google.com/github/saurav714/Simple-python-practice/blob/main/Copy_of_Building_a_Simple_AI_Agent_with_Open_AI%E2%80%99s_gpt_oss_20b_powered_by_NVIDIA_NIM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![image](https://i.imgur.com/CW48eG5.png)

# Building an AI Agent with OpenAI's gpt-oss-20b Powered by NVIDIA NIM

In the following notebook, we'll take a look at a few things:

1. How to use the NVIDIA-hosted NIM API to run inference on the model through the [Responses API](https://platform.openai.com/docs/api-reference/responses)
2. How to use the NVIDIA-hosted NIM API to run inference on the model through the [ChatCompletions API](https://platform.openai.com/docs/api-reference/chat)
3. How to build a Simple Web Search Agent powered by the `gpt-oss-20b` NIM using the [OpenAI Agents SDK](https://github.com/openai/openai-agents-python/tree/main).

Let's get right into it!

## Getting an API Key from build.nvidia.com

In order to use the NVIDIA-hosted NIM API - we'll need to have an API key from build.nvidia.com, luckily this is very straightforward.

First, let's navigate to the model on [build.nvidia.com](https://build.nvidia.com/openai/gpt-oss-20b)!

> NOTE: You'll need to ensure you're logged in before moving to the next steps!

Once there, you can click on the green button that says: `View Code`.

![image](https://i.imgur.com/mSGQPfC.png)

A new modal should appear on your screen, where you can click the `Generate API Key` text to obtain your API key!

![image](https://i.imgur.com/9ipSdPw.png)

Once you have that API key, you're good to move on to the next step which will capture it as an environment variable.

In [ ]:
import os
import getpass

os.environ["NVIDIA_API_KEY"] = getpass.getpass("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")

## Using the OpenAI Library with the NVIDIA-hosted NIM API API

We will be using the Python [OpenAI SDK](https://github.com/openai/openai-python) to access the `gpt-oss-20b` model on build.nvidia.com.

Let's start with a classic `pip install`.

In [ ]:
!pip install -qU openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 785.8/785.8 kB 22.2 MB/s eta 0:00:00


Once we've installed the `openai` library - we can use it to create an OpenAI client, which we'll point at the NVIDIA-hosted NIM API endpoint.

We'll also be sure to provide the API key we entered above by referencing the environment variable.

In [ ]:
from openai import OpenAI

client = OpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  api_key = os.environ["NVIDIA_API_KEY"]
)

Let's start with the Responses API which is powered by the new [Harmony](https://github.com/openai/harmony) response format.

> NOTE: This is something that NVIDIA NIM handles for users, so you can continue to use the APIs you're used to using without interruption.

Let's start with an easy question:

"How many 'r's in strawberry?".

Notice that through the Responses API we can control things like the model's reasoning effort just as you would expect!

> NOTE: Reasoning efforts are available in `low`, `medium`, and `high` for this model.

We're going to otherwise use the suggest defaults for our inference parameters, and set our `max_tokens` to a conservative `4096` - though the model itself supports a context length of up to `128K` tokens.

We'll also enable streaming, so we can see our streamed response as it's generated!

> NOTE: In the parsing of the response - we're able to specifically parse the reasoning vs. final output tokens!

In [ ]:
strawberry_prompt = """
How many 'r's in strawberry?
"""

response = client.responses.create(
  model="openai/gpt-oss-20b",
  input=[strawberry_prompt],
  reasoning={"effort" : "low"},
  max_output_tokens=4096,
  top_p=0.7,
  temperature=0.6,
  stream=True
)

reasoning_done = False
for chunk in response:
  if chunk.type == "response.reasoning_text.delta":
    print(chunk.delta, end="")
  elif chunk.type == "response.output_text.delta":
    if not reasoning_done:
      print("\n")
      reasoning_done = True
    print(chunk.delta, end="")

AuthenticationError: Error code: 401 - {'status': 401, 'title': 'Unauthorized', 'detail': 'Authentication failed'}

While the reasoning length may vary between runs - the model is able to accurately asses that there are 3 'r's in 'strawberry' in `low` reasoning mode.

Let's try a more difficult example in `high` reasoning mode.

We'll use an example from AIME25, a dataset designed to test reasoning model's Mathematics capabilities.

The example is as follows (small formatting tweaks for readability not present in the prompt):

```
The 9 members of a baseball team went to an ice cream parlor after their game.
Each player had a singlescoop cone of chocolate, vanilla, or strawberry ice cream.
At least one player chose each flavor, and the number of players who chose chocolate was greater than the number of players
who chose vanilla, which was greater than the number of players who chose strawberry.
Let $N$ be the number of different assignments of flavors to players that meet these conditions.
Find the remainder when $N$ is divided by 1000.
```

We expect that the model will arrive at the correct response of `16`.


In [ ]:
math_prompt = """
The 9 members of a baseball team went to an ice cream parlor after their game. Each player had a singlescoop cone of chocolate, vanilla, or strawberry ice cream. At least one player chose each flavor, and the number of players who chose chocolate was greater than the number of players who chose vanilla, which was greater than the number of players who chose strawberry. Let $N$ be the number of different assignments of flavors to players that meet these conditions. Find the remainder when $N$ is divided by 1000.

Please provide your answer in boxed format.
"""

response = client.responses.create(
  model="openai/gpt-oss-20b",
  input=[math_prompt],
  reasoning={"effort" : "high"},
  max_output_tokens=16384,
  top_p=0.7,
  temperature=0.6,
  stream=True
)

reasoning_done = False
for chunk in response:
  if chunk.type == "response.reasoning_text.delta":
    print(chunk.delta, end="")
  elif chunk.type == "response.output_text.delta":
    if not reasoning_done:
      print("\n")
      reasoning_done = True
    print(chunk.delta, end="")

Not only can we see that the model got the question correct - but we can also see an increase in the number of reasoning tokens used to arrive at the correct response!

## Building a Simple Web Search Agent Powered by the `gpt-oss-20b` NIM.

Next, we'll look at a very simple example of how we can build Agents leveraging the NVIDIA-hosted NIM API powered by NVIDIA NIM.

In order to get started, we need to grab both the `openai-agents`, and `tavily-python` library.

> NOTE: This example will require a Tavily API key, which you can obtain using the process outlined [here](https://docs.tavily.com/documentation/quickstart#get-your-free-tavily-api-key).

We will be using the ChatCompletion endpoint for this example due to the fact that the Responses API does not support tool-calling on the build.nvidia.com NVIDIA-hosted NIM API at this time.

> NOTE: The NIM container that you can download, deploy and run locally *does* support tool-calling in the Responses API. Instructions on how you can download and run the NIM are available [here](https://build.nvidia.com/openai/gpt-oss-20b/deploy)!

In [ ]:
!pip install -qU openai-agents tavily-python

In [ ]:
os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your TAVILY API key: ")

Enter your TAVILY API key: ··········


Next, we can create our `AsyncOpenAI` client through the same process we used for our `OpenAI` client.

In [ ]:
from openai import AsyncOpenAI

client = AsyncOpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  api_key = os.environ["NVIDIA_API_KEY"]
)

Since we need to search the web - we will create a `function_tool` that takes in a search query, and returns the results in a newline separated list.

In [ ]:
from agents import function_tool
from tavily import TavilyClient

tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

@function_tool
async def search_web(query: str) -> str:
    """Search the web for the given query.

    Args:
      query: The query to search for.
    """
    print(f"[INFO] Searching: {query}")
    response = tavily_client.search(query)
    return "\n".join([f"{result.title}: {result.content}" for result in response.results])

Next up, we'll create the actual Agent itself. You can read more about the OpenAI Agents SDK [here](https://github.com/openai/openai-agents-python/tree/main), we'll be focusing on the integration with the NVIDIA-hosted NIM API today.

The following parameter is what allows the integration to work smoothly.

```
OpenAIChatCompletionsModel(model="openai/gpt-oss-20b", ...)
```

Since NVIDIA NIM exposes the ChatCompletions API for the model - you can seamlessly integrate these models whether locally or through the NVIDIA-hosted NIM API.

In [ ]:
from agents import Agent, OpenAIChatCompletionsModel

agent = Agent(
        name="Assistant",
        instructions="You're a helpful assistant. You respond in a format that is useful for Enterprise Executives.",
        model=OpenAIChatCompletionsModel(model="openai/gpt-oss-20b", openai_client=client),
        tools=[search_web],
)

Now that we have our Agent, let's go ahead and *disable* automated tracing - since we're not providing our OpenAI key, and are only communicating with the NVIDIA-hosted NIM API powered by NIM on build.nvidia.com.

In [ ]:
from agents import set_tracing_disabled
set_tracing_disabled(disabled=True)

Finally, we can run our agent and see how it responds to a question about the enterprise benefits of NVIDIA NIM!

In [ ]:
from agents import Runner

result = await Runner.run(agent, "Briefly describe the enterprise benefits of NVIDIA NIM. 200 words or less.")
print(result.final_output)

[INFO] Searching: NVIDIA NIM enterprise benefits
[INFO] Searching: NVIDIA NIM enterprise benefits
**Enterprise benefits of NVIDIA NIM (NVIDIA Inference Model)** – <200 words  

1. **Rapid deployment** – NIMs are pre‑built, GPU‑optimized inference pipelines that let teams spin up large language, vision, or multimodal models in minutes, cutting time‑to‑market.  

2. **Scalable performance** – Built on NVIDIA’s Triton Inference Server and GPU‑cloud stack, NIMs auto‑scale with demand, delivering consistent low‑latency inference for millions of requests.  

3. **Cost efficiency** – Optimized kernels and mixed‑precision execution reduce GPU utilization and power consumption, lowering CAPEX and OPEX for on‑prem or cloud deployments.  

4. **Unified management** – A single API and dashboard control model lifecycle, monitoring, and logging, simplifying operations and reducing DevOps overhead.  

5. **Security & compliance** – NIMs run inside isolated containers, support role‑based access, and i

Now that you've built a simple Agent using NVIDIA NIM - head over the [model page](https://build.nvidia.com/openai/gpt-oss-20b) and try it out, or follow the [deployment instructions](https://build.nvidia.com/openai/gpt-oss-20b/deploy) to start building locally!